In [15]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup


import re, nltk, spacy, string

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

ModuleNotFoundError: No module named 'spacy'

In [2]:
df = pd.read_csv('data/modeling_ready_microwave1')

In [3]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        1875 non-null   int64  
 1   review_id          1875 non-null   object 
 2   product_id         1875 non-null   object 
 3   star_rating        1875 non-null   float64
 4   helpful_votes      1875 non-null   float64
 5   total_votes        1875 non-null   float64
 6   verified_purchase  1875 non-null   object 
 7   review_headline    1875 non-null   object 
 8   review_body        1875 non-null   object 
 9   review_date        1875 non-null   object 
 10  review_wordcount   1875 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 161.3+ KB


In [6]:
df.review_date = pd.to_datetime(df.review_date)

In [7]:
df.describe()

,customer_id,star_rating,helpful_votes,total_votes,review_wordcount
count,1.875000e+03,1875.000000,1875.000000,1875.000000,1875.000000
mean,3.209401e+07,3.850133,3.107733,3.628267,80.606933
std,1.476223e+07,1.496151,20.062047,20.926332,97.269705
min,8.722000e+04,1.000000,0.000000,0.000000,1.000000
25%,1.748780e+07,3.000000,0.000000,0.000000,28.000000
50%,3.303278e+07,5.000000,1.000000,1.000000,51.000000
75%,4.624525e+07,5.000000,2.000000,2.000000,94.000000
max,5.309426e+07,5.000000,589.000000,599.000000,1152.000000


In [ ]:
# n_features = 4000
# n_components = 20
# n_top_words = 20

# # ignore terms that have a document frequency strictly higher than 95%, 
# # ignore terms that have a document frequency strictly lower than 2
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
#                                    max_features=n_features,
#                                    stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(df_nouns['question_lemmatize_clean'].values.astype(str))

# # alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
# nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
# nmf_output = nmf.fit_transform(tfidf)

# def show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20):
#     keywords = np.array(vectorizer.get_feature_names())
#     topic_keywords = []
#     for topic_weights in lda_model.components_:
#         top_keyword_locs = (-topic_weights).argsort()[:n_words]
#         topic_keywords.append(keywords.take(top_keyword_locs))
#     return topic_keywords

# topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20)        

# # Topic - Keywords Dataframe
# df_topic_keywords = pd.DataFrame(topic_keywords)
# df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
# df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

# Topics_theme = ['Word start from ph', 'People/Friend/Relationship', 'Life/Experience/Love/Purpose', 'Money/Internet/Business', 
#                 'Weekend/Parent/Child', 'Leisure time', 'Language/technique/software', 'Relationship/Girl/Boy', 
#                 'Business relate to India, China or Pakistan', 'Friend/Love/Relationship', 'Difference and similarity/Language/Engineering', 
#                 'Culture, travel and visa requirements in several countries', 'Tips on working as software engineering', 'Book/Movie/Class/History/Physics/Chemistry/Science', 
#                 'Software engineer job opportunitis in Canada', 'Love/Life/Relationship', 'World/War/Language/History', 'Day/Hour/Week/Month/Sex/Place', 'School/Student/College/University', 
#                 'Question/Answer/Quora/Interview']
# df_topic_keywords['topic_theme'] = Topics_theme
# df_topic_keywords.set_index('topic_theme', inplace=True)
# df_topic_keywords.T